## A1-OpenJij (cxxjij編)

今までのチュートリアルではOpenJijの様々な機能を紹介してきましたが、本チュートリアルではOpenJijの更に深いレイヤーであるcxxjijの紹介をします。

基本的にOpenJijのチュートリアルを一通り読んだ人向けの内容で、

- 最適化問題だけでなくサンプリングなどより一般的な用途にOpenJijを用いたい
- アニーリングスケジュールの設定やアルゴリズム等を直接触りたい

といった目的に使用できます。

In [3]:
#!pip install openjij --user

In [2]:
!pip show openjij

Name: openjij
Version: 0.0.8
Summary: Framework for the Ising model and QUBO
Home-page: https://openjij.github.io/OpenJij/
Author: Jij Inc.
Author-email: openjij@j-ij.com
License: Apache License 2.0
Location: /home/jiko/.local/lib/python3.7/site-packages
Requires: numpy, debtcollector, requests
Required-by: 


## cxxjijについて

OpenJijはインターフェースはpythonで記述されていますが、内部はC++で実装されています。これらのC++インターフェースを直接呼び出しているインターフェースがcxxjijです。

今までチュートリアルで取り扱ってきたインターフェースはcxxjijを内部で呼び出す形で実装されています。

## Ising model 再訪

以前のチュートリアル (第一章)で紹介したIsingモデル

$$H(\{\sigma_i\}) = \sum_{i > j} J_{ij}\sigma_i \sigma_j + \sum_{i=1}^N h_i \sigma_i$$
$$\sigma_i \in \{-1, 1\}, i=1,\cdots N$$

の基底状態をcxxjijを用いて計算してみましょう。

第一章と同じく、変数の数が$N=5$で

$$h_i = -1~\text{for} ~\forall i, ~ J_{ij} = -1~\text{for} ~\forall i, j$$

の場合を考えます。この場合の基底状態は$\{\sigma_i\} = \{1, 1, 1, 1, 1\}$となります。

cxxjijによる実装では以下のような形式になります。

In [2]:
#cxxjijをインポートするには以下のように記述します。
import cxxjij as cj

#問題を表す縦磁場と相互作用を作ります。
#cxxjijでは専用のグラフを表すクラスを利用します。
import cxxjij.graph as G
N = 5

#密結合グラフを定義します。JijがスパースなモデルではSparseクラスを利用したほうが良いです。
J = G.Dense(N)

#Jij
for i in range(N):
    for j in range(i+1, N):
        J[i,j] = -1.0/N

#hi
for i in range(N):
    J[i] = -1
    #J[i,i] = -1でも同じになります。
    
#問題を解くsystemを定義します。今回は古典イジングスピンモデルを作成します。
import cxxjij.system as S
system = S.make_classical_ising_Eigen(J.gen_spin(), J)

#スケジュールリストを作成します。今回の例では、逆温度$\beta=0.1$から\beta=100まで、温度変化を100回に分けてアニーリングし、各温度で200モンテカルロステップ動かします。
import cxxjij.utility as U
schedule_list = U.make_classical_schedule_list(0.1, 100, 100, 200)

#アニーリングを行います。今回はSingleSpinFlipアルゴリズムを用いてsystemを更新します。
import cxxjij.algorithm as A
A.Algorithm_SingleSpinFlip_run(system, schedule_list)

#結果を取得します。
import cxxjij.result as R
result_spin = R.get_solution(system)

print(result_spin)

[1, 1, 1, 1, 1]


## cxxjijの解説

次に上で書いたコードの解説をします。cxxjijでは主に`graph, system, algorithm`のモジュールから成り立っており、それぞれのモジュールを組み合わせることで様々な種類、アルゴリズムを用いてイジングモデルを計算することが可能になっており、また新たにアルゴリズムを実装する際に拡張が容易であるという特徴を備えています。

### Graph

イジングハミルトニアンの係数$J_{ij}$を保持するためのモジュールです。基本的に密結合 (全てのJijが0以外の値を持つモデルに適している)を扱う`Dense`と疎結合 (Jijの多くの値が0であるモデルに適している)`Sparse`の2種類が存在します。また、`Sparse`から継承された`Square` (正方格子)、`Chimera` (D-Waveのキメラグラフ)も備えています。内部のC++実装ではそれぞれの構造に合わせて最適なデータ構造で実装されています。

### `Dense`、`Sparse`

`Dense`と`Sparse`の要素へのアクセス方法は次の通りです。

In [51]:
import cxxjij.graph as G

#密結合 (サイズ10)
dense = G.Dense(10)
#疎結合 (サイズ10)
sparse = G.Sparse(10)

#変数の書き込み
dense[3,8] = 4
dense[3,2] = 4
dense[3,5] = 3
#変数の読み出し
print("J_[3,8] = {}".format(dense[3,8]))
#インデックスはJij = Jjiとなるように設定されています。
print("J_[8,3] = {}".format(dense[8,3]))

#hiにアクセスするには次のようにインデックスを1つだけ指定します。
dense[4] = 10
#hiには、dense[4,4]のようにアクセスすることもできます。
print("J_[4,4] = {}".format(dense[4,4]))

#Sparseも基本的には同じです。

#変数の書き込み
sparse[3,8] = 5
sparse[3,2] = 4
sparse[5,3] = 2
#変数の読み出し
print("J_[3,8] = {}".format(sparse[3,8]))
#インデックスはJij = Jjiとなるように設定されています。
print("J_[8,3] = {}".format(sparse[8,3]))

J_[3,8] = 4.0
J_[8,3] = 4.0
J_[4,4] = 10.0
J_[3,8] = 5.0
J_[8,3] = 5.0


また、次の機能を備えています。

- .adj_nodes(i)
    - ノード番号iの隣接ノードを表示します。
- .gen_spin(seed)
    - ランダムなスピン列を生成します。seedが指定されない場合、ランダムなseedが設定されます
- .calc_energy(spin)
    - 与えられたspinのエネルギーを計算して、出力します。

In [54]:
#ノード3の隣接ノードは2,5,8です。
print(sparse.adj_nodes(3))

[8, 2, 5]


In [60]:
spin = sparse.gen_spin()
print(spin)
print(sparse.calc_energy(spin))

[-1, 1, -1, 1, 1, 1, -1, -1, -1, -1]
-7.0


### `Square`、`Chimera`

`Square`、`Chimera`は`Sparse`から継承されているクラスであり、先程の`Dense`や`Sparse`とは多少アクセス方法が異なります。

これら2つのクラスでは、座標と方向を指定することにより値にアクセスが可能となります。

`Square`では以下の図のように座標と方向を指定して値の設定を行います。

<img src="./square.png" width="500">

In [63]:
#正方格子 (サイズ10x10)
square = G.Square(10,10)

#値の指定 (座標(2,2)から(2,3)に向かう方向)
square[2,2,G.Dir.PLUS_C] = 3
#値の指定 (座標(2,2)から(3,2)に向かう方向)
square[2,2,G.Dir.PLUS_R] = 2
#値の指定 (座標(2,2)から(2,1)に向かう方向)
square[2,2,G.Dir.MINUS_C] = 1
#値の指定 (座標(2,2)から(1,2)に向かう方向)
square[2,2,G.Dir.MINUS_R] = 5
#値の指定 (座標(2,2)のhi)
square[2,2] = -1

#値の取得 (座標(2,3)から(2,2)に向かう方向には3が設定されている)
print(square[2,3,G.Dir.MINUS_C])

3.0


In [64]:
#Sparseへのキャスト
sparse = G.Sparse(square)
print(sparse[4])

0.0
